<a href="https://colab.research.google.com/github/PiKaChu-wcg/pytorch/blob/main/myGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.utils as vutil


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os
# 定义超参数
image_size = 28 #图像尺寸大小
input_dim = 100 #输入给生成器的向量维度，维度越大可以增加生成器输出样本的多样性
num_channels = 1# 图像的通道数
num_features = 64 #生成器中间的卷积核数量
batch_size = 64 #批次大小
use_cuda=torch.cuda.is_available() 
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!mkdir data
!tar -zxvf MNIST.tar.gz -C data

--2021-03-19 08:20:30--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-19 08:20:31--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz.2’

MNIST.tar.gz.2          [         <=>        ]  33.20M  4.68MB/s    in 19s     

2021-03-19 08:20:51 (1.73 MB/s) - ‘MNIST.tar.gz.2’ saved [34813078]

mkdir: cannot create directory ‘data’: File exists
MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-

In [23]:
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
itype = torch.cuda.LongTensor if use_cuda else torch.LongTensor

# 加载MINIST数据，如果没有下载过，就会在当前路径下新建/data子目录，并把文件存放其中
# MNIST数据是属于torchvision包自带的数据，所以可以直接调用。
# 在调用自己的数据的时候，我们可以用torchvision.datasets.ImageFolder或者torch.utils.data.TensorDataset来加载
train_dataset = dsets.MNIST(root='./data',  #文件存放路径
                            train=True,   #提取训练集
                            transform=transforms.ToTensor(),  #将图像转化为Tensor，在加载数据的时候，就可以对图像做预处理
                            download=True) #当找不到文件的时候，自动下载

# 加载测试数据集
test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# 训练数据集的加载器，自动将数据分割成batch，顺序随机打乱
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

'''我们希望将测试数据分成两部分，一部分作为校验数据，一部分作为测试数据。
校验数据用于检测模型是否过拟合，并调整参数，测试数据检验整个模型的工作'''


# 首先，我们定义下标数组indices，它相当于对所有test_dataset中数据的编码
# 然后定义下标indices_val来表示校验集数据的那些下标，indices_test表示测试集的下标
indices = range(len(test_dataset))
indices_val = indices[:5000]
indices_test = indices[5000:]

# 根据这些下标，构造两个数据集的SubsetRandomSampler采样器，它会对下标进行采样
sampler_val = torch.utils.data.sampler.SubsetRandomSampler(indices_val)
sampler_test = torch.utils.data.sampler.SubsetRandomSampler(indices_test)

# 根据两个采样器来定义加载器，注意将sampler_val和sampler_test分别赋值给了validation_loader和test_loader
validation_loader = torch.utils.data.DataLoader(dataset =test_dataset,
                                                batch_size = batch_size,
                                                sampler = sampler_val
                                               )
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          sampler = sampler_test
                                         )


In [24]:
class ModelD(nn.Module):
    def __init__(self):
        super(ModelD,self).__init__()
        self.model=nn.Sequential()
        self.model.add_module('conv1',nn.Conv2d(num_channels,num_features,5,2,0,bias=False))
        self.model.add_module('bnorm1',nn.BatchNorm2d(num_features))
        self.model.add_module('relu1',nn.ReLU())
        self.model.add_module('conv2',nn.Conv2d(num_features,num_features*2,5,2,0,bias=False))
        self.model.add_module('bnorm2',nn.BatchNorm2d(num_features*2))
        self.model.add_module('relu2',nn.ReLU())
        self.model.add_module('linear1',nn.Linear(num_features*2*4*4,num_features))
        self.model.add_module('linear2',nn.Linear(num_features,1))
        self.model.add_module('sigmoid',nn.Sigmoid())
    def forward(self,input):
        output=input
        for name,module in self.model.named_children():
            if name=='linear1':
                output=output.view(-1,num_features*2*4*4)
            output=module(output)
        return output
netD=ModelD()

In [25]:
netD

ModelD(
  (model): Sequential(
    (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (bnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (bnorm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (linear1): Linear(in_features=2048, out_features=64, bias=True)
    (linear2): Linear(in_features=64, out_features=1, bias=True)
    (sigmoid): Sigmoid()
  )
)

In [26]:
class ModelG(nn.Module):
    def __init__(self):
        super(ModelG,self).__init__()
        self.model=nn.Sequential() #model为一个内嵌的序列化的神经网络模型
        
        # 利用add_module增加一个反卷积层，输入为input_dim维，输出为2*num_features维，窗口大小为5，padding是0
        # 输入图像大小为1，输出图像大小为W'=(W-1)S-2P+K+P'=(1-1)*2-2*0+5+0=3, 5*5
        self.model.add_module('deconv1',nn.ConvTranspose2d(input_dim, num_features*2, 5, 2, 0, bias=False))
        # 增加一个batchnorm层
        self.model.add_module('bnorm1',nn.BatchNorm2d(num_features*2))
        # 增加非线性层
        self.model.add_module('relu1',nn.ReLU(True))
        # 增加第二层反卷积层，输入2*num_features维，输出num_features维，窗口5，padding=0
        # 输入图像大小为5，输出图像大小为W'=(W-1)S-2P+K+P'=(5-1)*2-2*0+5+0=13, 13*13
        self.model.add_module('deconv2',nn.ConvTranspose2d(num_features*2, num_features, 5, 2, 0, bias=False))
        # 增加一个batchnorm层
        self.model.add_module('bnorm2',nn.BatchNorm2d(num_features))
        # 增加非线性层
        self.model.add_module('relu2',nn.ReLU(True))

        # 增加第二层反卷积层，输入2*num_features维，输出num_features维，窗口4，padding=0
        # 输入图像大小为13，输出图像大小为W'=(W-1)S-2P+K+P'=(13-1)*2-2*0+4+0=28, 28*28
        self.model.add_module('deconv3',nn.ConvTranspose2d(num_features, num_channels, 4, 2, 0,bias=False))
        #self.model.add_module('tanh',nn.Tanh())
        self.model.add_module('sigmoid',nn.Sigmoid())
    def forward(self,input):
        output = input
        
        #遍历网络的所有层，一层层输出信息
        for name, module in self.model.named_children():
            output = module(output)
        #输出一张28*28的图像
        return(output)
netG=ModelG()

In [27]:
netG

ModelG(
  (model): Sequential(
    (deconv1): ConvTranspose2d(100, 128, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (bnorm1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (deconv2): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), bias=False)
    (bnorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU(inplace=True)
    (deconv3): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), bias=False)
    (sigmoid): Sigmoid()
  )
)

In [28]:
def weight_init(m):
    #模型参数初始化．
    #默认的初始化参数卷积核的权重是均值大概为0，方差在10^{-2}. BatchNorm层的权重均值是大约0.5，方差在0.2左右
    #使用如下初始化方式可以，可以让方差更小，使得收敛更快
    class_name=m.__class__.__name__
    if class_name.find('conv')!=-1:
        m.weight.data.normal_(0,0.02)
    if class_name.find('norm')!=-1:
        m.weight.data.normal_(1.0,0.02)
def make_show(img):
    # 将张量变成可以显示的图像
    img = img.data.expand(batch_size, 3, image_size, image_size)
    return img

def imshow(inp, title=None):
    # 在屏幕上绘制图像
    """Imshow for Tensor."""
    if inp.size()[0] > 1:
        inp = inp.numpy().transpose((1, 2, 0))
    else:
        inp = inp[0].numpy()
    mvalue = np.amin(inp)
    maxvalue = np.amax(inp)
    if maxvalue > mvalue:
        inp = (inp - mvalue)/(maxvalue - mvalue)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)

In [29]:
netD.apply(weight_init)
netG.apply(weight_init)
netD=netD.cuda() if use_cuda else netD
netG=netG.cuda() if use_cuda else netG
optimizerD=optim.Adam(netD.parameters(),lr=0.0005,betas=(0.5,0.999))
optimizerG=optim.Adam(netG.parameters(),lr=0.0005,betas=(0.5,0.999))
noise=torch.tensor((batch_size,input_dim,1,1),dtype=torch.float)
fixed_noise=torch.FloatTensor(batch_size,input_dim,1,1,).normal_(0,1).requires_grad_(True)
if use_cuda:
    noise=noise.cuda()
    fixed_noise=fixed_noise.cuda()
criterion=nn.BCELoss()
error_G=None
num_epochs=50
result=[]

In [33]:
for epoch in range(num_epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        optimizerD.zero_grad()
        data,target=data.clone().detach().requires_grad_(True),target.clone().detach()
        label=torch.ones(data.size()[0],1)
        if use_cuda:
            data,target,label=data.cuda(),target.cuda(),label.cuda()
        netD.train()
        output=netD(data)
        label.data.fill_(1)
        error_real=criterion(output,label)
        error_real.backward()
        D_x=output.data.mean()
        with torch.no_grad():
            noise.resize_(data.size()[0],input_dim,1,1).normal_(0,1)
        fake_pic=netG(noise).detach()
        output2=netD(fake_pic)
        label.data.fill_(0)
        error_fake=criterion(output2,label)
        error_fake.backward()
        error_D=error_real+error_fake
        optimizerD.step()
        if error_G is None or np.random.rand()<0.5:
            optimizerG.zero_grad()
            label.data.fill_(1)
            noise.data.normal_(0,1)
            netG.train()
            fake_pic=netG(noise)
            output=netD(fake_pic)
            error_G=criterion(output,label)
            error_G.backward()
            optimizerG.step()
        if use_cuda:
            error_D=error_D.cpu()
            error_G=error_G.cpu()
        result.append([float(error_D.data.numpy()),float(error_G.data.numpy())])
        if batch_idx % 100 == 0:
            print ('第{}周期，第{}/{}撮, 分类器Loss:{:.2f}, 生成器Loss:{:.2f}'.format(
                epoch,batch_idx,len(train_loader),
                error_D.data.item(), 
                error_G.data.item()))        
    netG.eval()
    fake_u=netG(fixed_noise)
    fake_u = fake_u.cpu() if use_cuda else fake_u
    img = make_show(fake_u)

    #挑选一些真实数据中的图像图像保存
    data, _ = next(iter(train_loader))
    os.makedirs('temp',exist_ok=True)
    os.makedirs('net',exist_ok=True)
    vutil.save_image(img,'temp/fake%s.png'% (epoch))
    # 保存网络状态到硬盘文件
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % ('net', epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % ('net', epoch))
    if epoch == 0:
        img = make_show(data.clone().detach().requires_grad_(True))
        vutil.save_image(img,'temp/real%s.png' % (epoch))

第0周期，第0/938撮, 分类器Loss:0.00, 生成器Loss:30.65
第0周期，第100/938撮, 分类器Loss:0.50, 生成器Loss:6.22
第0周期，第200/938撮, 分类器Loss:0.98, 生成器Loss:4.19
第0周期，第300/938撮, 分类器Loss:0.60, 生成器Loss:3.42
第0周期，第400/938撮, 分类器Loss:0.29, 生成器Loss:4.03
第0周期，第500/938撮, 分类器Loss:0.17, 生成器Loss:2.34
第0周期，第600/938撮, 分类器Loss:0.08, 生成器Loss:3.83
第0周期，第700/938撮, 分类器Loss:0.37, 生成器Loss:2.64
第0周期，第800/938撮, 分类器Loss:0.14, 生成器Loss:4.41
第0周期，第900/938撮, 分类器Loss:0.22, 生成器Loss:5.17
第1周期，第0/938撮, 分类器Loss:0.06, 生成器Loss:5.37
第1周期，第100/938撮, 分类器Loss:0.16, 生成器Loss:4.01
第1周期，第200/938撮, 分类器Loss:0.13, 生成器Loss:4.83
第1周期，第300/938撮, 分类器Loss:0.55, 生成器Loss:2.92
第1周期，第400/938撮, 分类器Loss:0.06, 生成器Loss:3.34
第1周期，第500/938撮, 分类器Loss:0.33, 生成器Loss:7.79
第1周期，第600/938撮, 分类器Loss:0.27, 生成器Loss:6.18
第1周期，第700/938撮, 分类器Loss:0.17, 生成器Loss:4.72
第1周期，第800/938撮, 分类器Loss:0.11, 生成器Loss:6.67
第1周期，第900/938撮, 分类器Loss:0.47, 生成器Loss:3.04
第2周期，第0/938撮, 分类器Loss:0.16, 生成器Loss:5.52
第2周期，第100/938撮, 分类器Loss:0.41, 生成器Loss:2.30
第2周期，第200/938撮, 分类器Loss:0.05, 生成器Loss:4.64
第2周期，第300/938撮, 

In [ ]:
!pwd

In [ ]:
!ls